LangChainMemory 
```
이전 대화 내용을 기억해서 문맥을 유지하는 역할
LangChain 0.3x부터는 LCEL 기반으로 체인을 구성
RunnableWithMessageHistory , ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화 기록을 관리
대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결
```

In [4]:
%pip install --quiet langchain langchain-openai python-dotenv langchain_redis

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

human : 안녕하세요 제 이름은 홍길동 입니다.
ai : 안녕하세요 홍길동님, 무엇을 도와드릴까요?


In [5]:
# Radis 기반 채팅 기록 저장소
from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv('REDIS_URL', 'redis://localhost:6379')
session_id = 'user_123'
history = RedisChatMessageHistory(session_id=session_id, redis_url=REDIS_URL)
history.add_user_message("안녕하세요 제 이름은 홍길동 입니다.")
history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

ConnectionError: Error 61 connecting to localhost:6379. Connection refused.

In [6]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗복
# 핵심: session_id를 키로 하는 메모리 저장소 만들고 사용자의 대화는 키별로 저장한다
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system",'당신은 뛰어난 한국어 상담 챗봇입니다. 질문에 친절하고 자세히 답변해주세요'),
    #history 키로 전달된 메세지 목록은 chain 실행시 해당 위치에 넣겠다는 의미
    MessagesPlaceholder(variable_name='history'), 
    ('human','{input}')
])
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [ ]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [8]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_session_history를 구현

In [19]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id:str) -> InMemoryChatMessageHistory:
    '''"세션 ID"에 해당하는 대화 기록 객체를 반환합니다. (없으면 바로 생성) '''
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='history'
)

In [20]:
# 두개의 세션을 번갈아가면서 대화 RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다
sessions = ['user_a','user_b']
questions = [
    '안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?', #user_a 첫번째 질문
    '안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?', #user_b 첫 번째 질문
    '저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?', #user_a 두 번째 질문
    '저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?' #user_b 두 번째 질문
]
for i,question in enumerate(questions):
    session_id = sessions[i%2] # 세션 ID를 번갈아가며 사용 
    result = chatbot.invoke({'input':question}, config = {'configurable': {'session_id':session_id}})
    print(f'[{session_id}] 질문: {question}')
    print(f'[{session_id}] 챗봇: {result}\n')

16:34:24 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_a] 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
[user_a] 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

16:34:26 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_b] 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
[user_b] 챗봇: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 다양한 정보와 도움을 제공하는 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

16:34:27 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_a] 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
[user_a] 챗봇: 프로그래밍을 배우고 계시다니 멋지네요! 저는 여러분의 질문에 답변하고, 정보 제공, 문제 해결, 그리고 다양한 주제에 대한 상담을 하는 역할을 하고 있습니다. 프로그래밍에 관련된 질문이나 도움이 필요하시면 언제든지 말씀해 주세요. 어떤 언어를 배우고 계신가요?

16:34:29 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_b] 질문: 저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?
[user_b] 챗봇: 역사에 관심이 많으시다니 정말 멋지